In [105]:
import pandas as pd
import numpy as np
def timetrans(a):
    return a[0:4]+'-'+a[5:7]+'-'+a[8:10]

info=pd.read_csv('sku_info.csv')
info = pd.DataFrame(info.values, columns=['sku_id','first','second','third','brand'])

sales = pd.read_csv('sku_sales.csv')
sales.columns = ['sku_id','dc_id','date','quantity','vend','or_price','discount']
#sales.loc[:,'date']=sales.date.map(timetrans)

attr=pd.read_csv('sku_attr.csv')
attr = pd.DataFrame(attr.values, columns=['sku_id','attr','attr_values'])

prom=pd.read_csv('sku_prom.csv')
prom = pd.DataFrame(prom.values, columns=['date','sku_id','third','prom_type'])
prom.loc[:,'date']=prom.date.map(timetrans)

test_prom=pd.read_csv('sku_prom_testing_2018Jan.csv')
test_prom=pd.DataFrame(test_prom.values,columns=['date','sku_id','third','prom_type'])
test_prom.loc[:,'date']=test_prom.date.map(timetrans)

a=info.drop(['first','second','brand'],axis=1)
prom09=prom[prom.sku_id!=-999]
b=prom[prom.sku_id==-999].drop(['sku_id'],axis=1)
prom999=pd.merge(b,a,how='left',on='third')
prom_ok=pd.concat([prom999,prom09])

In [106]:
business=pd.merge(sales,prom_ok,how='left',on=['sku_id','date'])

In [107]:
print sales.shape
print prom_ok.shape
print business.shape

(2127485, 7)
(862250, 4)
(3842160, 9)


In [6]:
print prom_ok.groupby(['sku_id','date'])['prom_type'].unique()

KeyboardInterrupt: 

In [108]:
a=pd.get_dummies(prom_ok['prom_type'],prefix='prom')
b=pd.concat([prom_ok,a],axis=1)
c=pd.pivot_table(b,index=['date','sku_id','third'],aggfunc=np.sum).reset_index()

In [8]:
print c[(c.prom_1!=0)&(c.prom_4!=0)&(c.prom_6!=0)&(c.prom_10!=0)&(c.sku_id==637)]

              date  sku_id  third  prom_1  prom_10  prom_4  prom_6
5609    2016-01-13     637     17       1        1       1       1
6067    2016-01-14     637     17       1        1       1       1
6529    2016-01-15     637     17       1        1       1       1
6991    2016-01-16     637     17       1        1       1       1
7453    2016-01-17     637     17       1        1       1       1
7905    2016-01-18     637     17       1        1       1       1
8360    2016-01-19     637     17       1        1       1       1
8821    2016-01-20     637     17       1        1       1       1
9279    2016-01-21     637     17       1        1       1       1
9747    2016-01-22     637     17       1        1       1       1
10218   2016-01-23     637     17       1        1       1       1
10688   2016-01-24     637     17       1        1       1       1
11160   2016-01-25     637     17       1        1       1       1
21905   2016-02-17     637     17       1        1       1    

In [9]:
sales_prom=pd.merge(sales,c,on=['sku_id','date'],how='left').drop(['third'],axis=1)

In [10]:
table3=pd.merge(sales_prom,info,on=['sku_id'],how='left')


In [12]:
print table3[table3.prom_10>=2]

         sku_id  dc_id        date  quantity  vend  or_price   discount  \
1272        683      1  2016-05-28      20.0   1.0  0.021003   6.389881   
2353        560      1  2016-05-26       0.0   0.0       NaN        NaN   
7836        617      0  2016-05-30      26.0   1.0  0.031129   7.971888   
26152       467      3  2016-05-25       0.0   1.0       NaN        NaN   
30032       707      1  2016-05-14       0.0   1.0       NaN        NaN   
31683       665      1  2016-05-30       0.0   1.0       NaN        NaN   
33266       560      0  2016-05-23       0.0   0.0       NaN        NaN   
33907       778      2  2016-05-27       0.0   0.0       NaN        NaN   
44175       937      0  2016-05-17       0.0   0.0       NaN        NaN   
48385       674      2  2016-05-13       0.0   1.0       NaN        NaN   
50476       467      3  2016-05-23       0.0   1.0       NaN        NaN   
53434       683      0  2016-05-25      50.0   1.0  0.021003   6.442857   
55492       560      1  2

In [109]:


b=attr.groupby(['sku_id'])['attr'].unique().reset_index()
b.loc[:,'attr']=b['attr'].map(lambda x: len(x))
a=b.copy()

b=attr.groupby(['sku_id'])['attr_values'].unique().reset_index()
b.loc[:,'attr_values']=b['attr_values'].map(lambda x: len(x))
a=pd.merge(a,b,how='left',on='sku_id')

a['values_rate']=a['attr_values']/a['attr']

table4=pd.merge(table3,a,how='left',on='sku_id')
print table4

         sku_id  dc_id        date  quantity  vend  or_price   discount  \
0           637      0  2016-10-12       5.0   1.0  0.089886   7.858136   
1           637      0  2017-09-24       1.0   1.0  0.089886   8.191933   
2           637      3  2016-07-01       0.0   1.0       NaN        NaN   
3           637      3  2016-08-03       2.0   1.0  0.089886   8.122392   
4           637      3  2017-05-03       4.0   1.0  0.089886   8.136300   
5           937      1  2016-02-12       0.0   0.0       NaN        NaN   
6           937      1  2017-02-01       0.0   0.0       NaN        NaN   
7           937      1  2017-03-25       0.0   0.0       NaN        NaN   
8           937      1  2017-04-24       0.0   1.0       NaN        NaN   
9           937      4  2017-12-14       0.0   1.0       NaN        NaN   
10          937      5  2017-12-28       0.0   1.0       NaN        NaN   
11          937      0  2016-01-14       2.0   1.0  0.014877   5.000000   
12          937      2  2

In [114]:

###test添加attr属性
print a.head()
test=pd.read_csv('test.csv')
test.drop(test.columns[[0]], axis=1,inplace=True)
uu=pd.merge(test,a,how='left',on=['sku_id'])
print uu.tail()
#uu.to_csv('test.csv')

   sku_id  attr  attr_values  values_rate
0       1     2            6          3.0
1       2     1            1          1.0
2       3     1            1          1.0
3       4     4            2          0.5
4       5     4            4          1.0
              date  dc_id  sku_id  prom_1  prom_10  prom_4  prom_6  first  \
185995  2018-01-31      5     996     1.0      1.0     0.0     0.0    1.0   
185996  2018-01-31      5     997     1.0      0.0     0.0     0.0    7.0   
185997  2018-01-31      5     998     0.0      1.0     0.0     0.0    1.0   
185998  2018-01-31      5     999     NaN      NaN     NaN     NaN    NaN   
185999  2018-01-31      5    1000     0.0      1.0     0.0     0.0    1.0   

        second  third   brand  attr  attr_values  values_rate  
185995     4.0   80.0   225.0     5            5     1.000000  
185996    31.0  358.0  1153.0     7            7     1.000000  
185997     7.0  298.0   158.0     1            1     1.000000  
185998     NaN    NaN     NaN

In [49]:
print table3.shape
print table4.shape
print sales.shape
table4.to_csv('train.csv')

(2127485, 15)
(2127485, 18)
(2127485, 7)


In [51]:
attr['count']=pd.Series(1, index=attr.index)
bbb=attr.groupby(['sku_id','attr','attr_values'])['count'].sum()
print bbb

sku_id  attr  attr_values
1       1     1433           1
              1523           1
              1524           1
              1525           1
              1528           1
        2     148            1
2       60    2436           1
3       82    1298           1
4       68    1263           1
        390   1263           1
        488   433            1
        584   1263           1
5       22    174            1
        37    299            1
        103   1194           1
        277   1553           1
6       42    20             1
        765   1435           1
        876   4              1
        877   4582           1
        878   4583           1
        879   4584           1
              4585           1
        880   4586           1
        881   1447           1
7       50    2065           1
        68    2054           1
        82    1298           1
        225   409            1
        390   2062           1
                            ..
994     390  

In [52]:
aa=info.drop(['first','second','brand'],axis=1)
test_prom09=test_prom[test_prom.sku_id!=-999]
bb=test_prom[test_prom.sku_id==-999].drop(['sku_id'],axis=1)
test_prom999=pd.merge(bb,aa,how='left',on='third')
test_prom_ok=pd.concat([test_prom999,test_prom09])
print test_prom_ok

             date prom_type sku_id third
0      2018-01-01        10      1   366
1      2018-01-02        10      1   366
2      2018-01-03        10      1   366
3      2018-01-04        10      1   366
4      2018-01-05        10      1   366
5      2018-01-06        10      1   366
6      2018-01-07        10      1   366
7      2018-01-08        10      1   366
8      2018-01-09        10      1   366
9      2018-01-10        10      1   366
10     2018-01-11        10      1   366
11     2018-01-12        10      1   366
12     2018-01-13        10      1   366
13     2018-01-14        10      1   366
14     2018-01-15        10      1   366
15     2018-01-16        10      1   366
16     2018-01-17        10      1   366
17     2018-01-18        10      1   366
18     2018-01-19        10      1   366
19     2018-01-20        10      1   366
20     2018-01-21        10      1   366
21     2018-01-22        10      1   366
22     2018-01-23        10      1   366
23     2018-01-1

In [53]:
a=pd.get_dummies(test_prom_ok['prom_type'],prefix='prom')
b=pd.concat([test_prom_ok,a],axis=1)
ccc=pd.pivot_table(b,index=['date','sku_id','third'],aggfunc=np.sum).reset_index()

In [55]:
test1= pd.merge(ccc.drop(['third'],axis=1),info,on=['sku_id'],how='left')

In [57]:
print test1

             date  sku_id  prom_1  prom_10  prom_4  prom_6  first  second  \
0      2018-01-01       1       0        1       0       0      1       5   
1      2018-01-01       2       0        1       0       0      1       7   
2      2018-01-01       3       0        1       0       0      7      29   
3      2018-01-01       4       1        0       0       0      7      29   
4      2018-01-01       5       1        1       0       0      1       4   
5      2018-01-01       6       1        0       0       0      1       3   
6      2018-01-01       7       1        1       0       1      7      29   
7      2018-01-01       8       0        1       0       0      7      30   
8      2018-01-01       9       0        1       0       0      7      29   
9      2018-01-01      10       1        1       0       0      1       4   
10     2018-01-01      11       1        0       0       0      7      30   
11     2018-01-01      13       1        0       0       0      7      29   

In [68]:
for i in table4:
    print i,table4[i].dtype

 sku_id int64
dc_id int64
date object
quantity float64
vend float64
or_price float64
discount float64
prom_1 float64
prom_10 float64
prom_4 float64
prom_6 float64
first int64
second int64
third int64
brand int64
attr int64
attr_values int64
values_rate float64


In [100]:
#a = pd.DataFrame(columns=['date', 'dc_id', 'sku_id'])
a=np.zeros((186000, 3))
print a.shape
uu=0
for i in range(31):
    for j in range(6):
        for k in range(1,1000+1):
            a[uu,:]=[i,int(j),int(k)]
            uu+=1
a=pd.DataFrame(a,columns=['date', 'dc_id', 'sku_id'])
qq=test1.date.unique()
a.loc[:,'date']=a['date'].map(lambda x: qq[int(x)] )
a.loc[:,'dc_id']=a['dc_id'].map(lambda x: int(x))
a.loc[:,'sku_id']=a['sku_id'].map(lambda x: int(x) )
print a

(186000L, 3L)
              date  dc_id  sku_id
0       2018-01-01      0       1
1       2018-01-01      0       2
2       2018-01-01      0       3
3       2018-01-01      0       4
4       2018-01-01      0       5
5       2018-01-01      0       6
6       2018-01-01      0       7
7       2018-01-01      0       8
8       2018-01-01      0       9
9       2018-01-01      0      10
10      2018-01-01      0      11
11      2018-01-01      0      12
12      2018-01-01      0      13
13      2018-01-01      0      14
14      2018-01-01      0      15
15      2018-01-01      0      16
16      2018-01-01      0      17
17      2018-01-01      0      18
18      2018-01-01      0      19
19      2018-01-01      0      20
20      2018-01-01      0      21
21      2018-01-01      0      22
22      2018-01-01      0      23
23      2018-01-01      0      24
24      2018-01-01      0      25
25      2018-01-01      0      26
26      2018-01-01      0      27
27      2018-01-01      0      28


In [103]:
test2=pd.merge(a,test1,how='left',on=['date','sku_id'])
print test2

              date  dc_id  sku_id  prom_1  prom_10  prom_4  prom_6  first  \
0       2018-01-01      0       1     0.0      1.0     0.0     0.0    1.0   
1       2018-01-01      0       2     0.0      1.0     0.0     0.0    1.0   
2       2018-01-01      0       3     0.0      1.0     0.0     0.0    7.0   
3       2018-01-01      0       4     1.0      0.0     0.0     0.0    7.0   
4       2018-01-01      0       5     1.0      1.0     0.0     0.0    1.0   
5       2018-01-01      0       6     1.0      0.0     0.0     0.0    1.0   
6       2018-01-01      0       7     1.0      1.0     0.0     1.0    7.0   
7       2018-01-01      0       8     0.0      1.0     0.0     0.0    7.0   
8       2018-01-01      0       9     0.0      1.0     0.0     0.0    7.0   
9       2018-01-01      0      10     1.0      1.0     0.0     0.0    1.0   
10      2018-01-01      0      11     1.0      0.0     0.0     0.0    7.0   
11      2018-01-01      0      12     NaN      NaN     NaN     NaN    NaN   

In [104]:
test2.to_csv('test.csv')